In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source=dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date=dbutils.widgets.get("p_file_date")

#### Describe Schema

In [0]:
constructor_schema="constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"


#### Apply Schema

In [0]:
df=spark.read.schema(constructor_schema).json(f"{raw_folder_path}/{v_file_date}/constructors.json")

#### DROP the url column

In [0]:
from pyspark.sql.functions import col, lit, current_timestamp
dropped_df=df.drop("url")

#### Renaming the column

In [0]:
renamed_df=dropped_df.withColumnRenamed("constructorId","constructor_id")\
.withColumnRenamed("constructorRef","constructor_ref")\
.withColumn("data_source",lit(v_data_source))\
.withColumn("file_date",lit(v_file_date))                

#### Add Ingestion Date Column

In [0]:
final_df=add_ingestion_date(renamed_df)
final_df.display()

#### Write Data

In [0]:
final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.constructors")